In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
from time import time

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from GetAgentData import *

<h2>Variational Autoencoder</h2>

<p>An autoencoding model that takes an input, in this case it will be the load condtions and a part image, and produces a latentspace, in terms of mean and standard deviation. We then use a decoder to expand this latent space on a specific normal distribution to regenerat the optimal part.</p>
<p>The input space for this model would be an image of a current part(as a seed) the load conditions could be kept as either a vector or could be expanded into an image like the iterative model. The latent space would have to be standardized as would the image resolutions. The output would just be the final iteration of the part.</p>
</div>
<p>The second stage to this setup would have us take the decoder and train a new encoder to produce the same latent space but with only the format vector and a noise value. This would then have to be pluged into the decoder to produce the optimal part.</p>
<p>A special training regime would have to be run for this setup. The load encoder would produce a set of means and standard deviations. The mean of this set(no deviation) should decode into the optimal part for the load conditions. The standard deviations can be added to get some similar parts around the optimal.</p>



In [ ]:
#overview of the data
#dataDirectory = os.path.join(os.getcwd(),'data')
dataDirectory = r"E:\TopoptGAfileSaves\ComplianceMinimization\Agents"
DATA_FILE_PATH = os.path.join(dataDirectory,'100_50')

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])

